In [ ]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")
    
    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/workshops')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


# CHSH Bell Inequality - Hands-On Workshop

**Learning Objectives:**
- Create entangled Bell states  
- Implement measurement-basis transformations  
- Calculate correlations and the CHSH value  
- Verify that quantum entanglement violates classical limits  

**Goal:** Demonstrate that entangled pairs can produce $|S| > 2$.


**Roadmap (functions you’ll implement):**
`create_bell_pair_singlet_state()` → `apply_basis_transformation()` → `measure_bell_pair()` → `organize_measurements_by_basis()` → `calculate_correlations()` → `calculate_chsh_value()`

---

## CHSH Bell Inequality: Key Points (Quick Reference)

The CHSH test checks whether two systems (e.g., qubits) are **quantum entangled**.

### What the CHSH value means

| System | CHSH Value $\|S\|$ |
|--------|-------------|
| Classical (hidden variables) | $\|S\| \le 2$ |
| **Quantum (entangled)** | **$2 < \|S\| \le 2\sqrt{2} \approx 2.83$** |

If $|S| > 2$ → **Quantum entanglement confirmed!**

In practice, a statistically significant violation (beyond the error bars) is required to experimentally conclude that the state is entangled.


### Which measurement bases are used?

The test uses **four combinations** of measurement bases for Alice and Bob. Each side uses **two bases**:
- **Alice:** $a_1 = 0^\circ$, $a_2 = 90^\circ$
- **Bob:** $b_1 = 45^\circ$, $b_2 = 135^\circ$

Pairs used in CHSH: $(a_1,b_1)$, $(a_1,b_2)$, $(a_2,b_1)$, $(a_2,b_2)$

Bases are applied by `apply_basis_transformation()` and measured in `measure_bell_pair()`.

### Correlation for each basis pair
Computed by `calculate_correlations()`
$$
E(a,b) = \frac{N_{00} + N_{11} - N_{01} - N_{10}}{N_{00} + N_{11} + N_{01} + N_{10}}
$$

Where $N_{xy}$ is the **count** of measurement outcomes $x$ for Alice and $y$ for Bob (e.g., $N_{00}$ is how many times the result was $00$).

Counts grouped by `organize_measurements_by_basis()`.

### CHSH formula

$$
S = E(a_1,b_1) - E(a_1,b_2) + E(a_2,b_1) + E(a_2,b_2)
$$

Computed by `calculate_chsh_value()`.

---

## Bridge between the presentation and the notebook: physics → simulation

**Real experiment:** entangled photons + polarizers/waveplates that set the measurement angles.  
**In practice:** we rotate optical elements to choose the measurement basis.  
**In this notebook:** Bell pairs + rotation gates to simulate those angles in Qiskit.

So when you see **0°, 45°, 90°, 135°** below, think **polarizer angles** in the lab.

---


## Setup

Run this cell to import all required libraries and helper functions.

In [1]:
# Standard imports
import random
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Set seed for reproducibility
GLOBAL_SEED = 91
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# Initialize simulator
aer_simulator = AerSimulator()
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

# Constants
ALICE_BELL_BASES = ['0', '90']   # Alice's measurement bases for CHSH
BOB_BELL_BASES = ['45', '135']   # Bob's measurement bases for CHSH
BELL_INEQUALITY_THRESHOLD = 2.0  # Classical limit

print("Setup complete!")
print(f"Alice's bases: {ALICE_BELL_BASES}")
print(f"Bob's bases: {BOB_BELL_BASES}")

Setup complete!
Alice's bases: ['0', '90']
Bob's bases: ['45', '135']


In [2]:
# Helper function to run circuits (provided - just run this cell)
def run_circuit(circ: QuantumCircuit, shots=1) -> dict:
    """
    Run a quantum circuit on the AerSimulator and return the counts.
    """
    global MANUAL_SIMULATOR_SEED_COUNTER
    global aer_simulator
    
    current_run_seed = MANUAL_SIMULATOR_SEED_COUNTER
    MANUAL_SIMULATOR_SEED_COUNTER += 1
    
    circ = transpile(circ, aer_simulator)
    result = aer_simulator.run(circ, shots=shots, seed_simulator=current_run_seed).result()
    return result.get_counts(circ)

print("Helper function loaded!")

Helper function loaded!


---
## 1) Create a Bell State

We start by preparing an **entangled pair**. The singlet state guarantees perfect anti‑correlation, which is what gives a CHSH violation.

The **singlet state** $|\Psi^-\rangle = (|01\rangle - |10\rangle)/\sqrt{2}$ is used in the original E91 protocol.

**Recipe:**
1. Apply Hadamard (H) to qubit 0
2. Apply CNOT with control=0, target=1
3. Apply X then Z to qubit 1 (to get $|\Psi^-\rangle$ specifically)

Exercise: Implement `create_bell_pair_singlet_state()`

In [ ]:
def create_bell_pair_singlet_state() -> QuantumCircuit:
    """
    Create the Bell singlet state |Ψ-⟩ = (|01⟩ - |10⟩)/√2
    
    Returns:
        QuantumCircuit with 2 qubits in the singlet Bell state
    """
    # TODO: Create a quantum circuit with 2 qubits
    qc = None  # Replace with: QuantumCircuit(2)
    
    # Step 1: Create superposition
    # TODO: Apply Hadamard to qubit 0
    
    # Step 2: Entangle qubits
    # TODO: Apply CNOT (control=0, target=1)
    
    # Step 3: Transform to |Ψ-⟩ state
    # TODO: Apply X gate to qubit 1
    # TODO: Apply Z gate to qubit 1
    
    return qc

In [ ]:
# Test your Bell state
bell_state = create_bell_pair_singlet_state()
bell_state.draw('mpl')

In [ ]:
# Verify by measuring many times
test_qc = bell_state.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)
print(f"\nMeasurement results: {counts}")
print("\nExpected: roughly 50% |01⟩ and 50% |10⟩ (never |00⟩ or |11⟩)")


---
## 2) Measurement Basis Transformation

We always measure in the Z‑basis, so to simulate other bases we **rotate first**, then measure.

| Basis | Angle | Gate |
|-------|-------|------|
| Z-basis | 0° | None |
| 45° | 45° | Ry(-π/4) |
| X-basis | 90° | H |
| 135° | 135° | Ry(-3π/4) |

Exercise: Implement `apply_basis_transformation()`

### Quick intuition (bases)

- **Same basis** → strong (anti-)correlation for entangled pairs.
- **Different bases** → results look more random.

This is exactly what the polarizer angles do in the photon experiment.

---

In [ ]:
def apply_basis_transformation(circuit: QuantumCircuit, qubit_index: int, basis: str) -> QuantumCircuit:
    """
    Apply a measurement basis transformation to a qubit.
    
    Args:
        circuit: The quantum circuit to transform
        qubit_index: Index of the qubit (0 or 1)
        basis: '0' (Z), '45', '90' (X), or '135'
    
    Returns:
        Copy of the circuit with basis transformation applied
    """
    # Create a copy of the circuit
    transformed = circuit.copy()
    
    if basis == '0':    # Z-basis (0°)
        pass  # No transformation needed
    
    elif basis == '90':  # X-basis (90°)
        # TODO: Apply Hadamard gate on the specified qubit (qubit_index)
        pass
    
    elif basis == '45':  # 45° basis
        # TODO: Apply Ry(-π/4) rotation
        # Hint: transformed.ry(-np.pi/4, qubit_index)
        pass
    
    elif basis == '135':  # 135° basis
        # TODO: Apply Ry(-3π/4) rotation
        pass
    
    else:
        raise ValueError(f"Unknown basis: {basis}")
    
    return transformed

In [ ]:
#  Test basis transformations
test_qc = QuantumCircuit(1)

print("Testing basis transformations:")
for basis in ['0', '45', '90', '135']:
    result = apply_basis_transformation(test_qc, 0, basis)
    print(f"\nBasis {basis}°:")
    print(result.draw())

---
## 3) Measure Bell Pairs

Each entangled pair can be measured **only once** because measurement collapses the state.

Now we combine Bell state creation with basis transformations to measure entangled pairs.

Exercise: Implement `measure_bell_pair()`

In [ ]:
def measure_bell_pair(circuit: QuantumCircuit, alice_basis: str, bob_basis: str) -> str:
    """
    Measure a Bell pair with Alice and Bob using specified bases.
    
    Args:
        circuit: Bell pair circuit
        alice_basis: Alice's measurement basis ('0', '45', '90')
        bob_basis: Bob's measurement basis ('45', '90', '135')
    
    Returns:
        str: Measurement result ('00', '01', '10', or '11')
    """
    # Create a copy of the circuit
    meas_qc = circuit.copy()
    
    # TODO: Apply Alice's basis transformation to qubit 0
    
    # TODO: Apply Bob's basis transformation to qubit 1
    
    # Add measurements to all qubits
    meas_qc.measure_all()
    
    # Run the circuit and return the result
    counts = run_circuit(meas_qc, shots=1)
    
    return list(counts.keys())[0]

In [ ]:
# Test Bell pair measurement
bell_state = create_bell_pair_singlet_state()
from collections import Counter


# Test multiple measurements
print("Testing Bell pair measurements (1000 runs each)")

print("\nDifferent bases (Alice 0°, Bob 45°) - Random results:")
results_diff = [measure_bell_pair(bell_state, '0', '45') for _ in range(200)]
print(f"  Counts: {dict(Counter(results_diff))}")

print("\nSame bases (Alice 0°, Bob 0°) - Perfect Anti-correlation:")
# For Singlet State |Ψ-⟩, same-basis measurements must be opposite ('01' or '10')
results_same = [measure_bell_pair(bell_state, '0', '0') for _ in range(200)]
print(f"  Counts: {dict(Counter(results_same))}")


---
## 4) Run Bell Test Measurements

To estimate CHSH, we need many trials with **randomly chosen bases** to sample all four basis pairs.

For the CHSH test, we need to:
1. Create many Bell pairs
2. Randomly choose bases for Alice and Bob
3. Record all measurements

Exercise: Implement `run_bell_test_measurements()`

In [ ]:
def run_bell_test_measurements(list_bell_pairs, list_alice_bases=ALICE_BELL_BASES, list_bob_bases=BOB_BELL_BASES):
    """
    Run measurements on Bell pairs using random basis choices.
    
    Args:
        list_bell_pairs: List of Bell pair circuits
        list_alice_bases: Alice's possible bases
        list_bob_bases: Bob's possible bases
    
    Returns:
        Tuple of (results, alice_bases_used, bob_bases_used)
    """
    results = []
    alice_bases_used = []
    bob_bases_used = []
    
    for qc in list_bell_pairs:
        # TODO: Randomly choose Alice's basis from list_alice_bases
        a_base = None  # Hint: random.choice(list_alice_bases)
        
        # TODO: Randomly choose Bob's basis from list_bob_bases
        b_base = None # Hint: random.choice(list_bob_bases)
        
        # TODO: Measure the Bell pair qc with a_base and b_base
        result = None  # Hint: measure_bell_pair(qc, a_base, b_base)
        
        # Record everything
        results.append(result)
        alice_bases_used.append(a_base)
        bob_bases_used.append(b_base)
    
    return results, alice_bases_used, bob_bases_used

---
## 5) Organize Results by Basis Pair

We must **group results by basis pair** so we can compute $E(a,b)$ for each combination.



In [ ]:
def organize_measurements_by_basis(results, alice_bases, bob_bases):
    """
    Organize measurements by basis pairs.
    
    Returns:
        Dict mapping (alice_basis, bob_basis) to {'00': count, '01': count, ...}
    """
    # Get unique bases
    unique_alice = list(set(alice_bases))
    unique_bob = list(set(bob_bases))
    
    # Initialize counts dictionary
    counts = {}
    for a in unique_alice:
        for b in unique_bob:
            counts[(a, b)] = {'00': 0, '01': 0, '10': 0, '11': 0}
    
    # Loop through results and increment the counter for the used basis pair
    for i, result in enumerate(results):
        a_base = alice_bases[i]
        b_base = bob_bases[i]
        
        # Check if result is valid (just in case), then increment
        if result in counts[(a_base, b_base)]:
            counts[(a_base, b_base)][result] += 1
            
    return counts


---
## 6) Calculate Correlations

Correlation tells us whether Alice and Bob are **usually the same or different**. It always lies between **−1 and +1**.

**Counts formula (what we use in code):**
$$
E(a,b)=\frac{N_{00}+N_{11}-N_{01}-N_{10}}{N_{00}+N_{11}+N_{01}+N_{10}}
$$

### Link to the slide definition (±1 outcomes)

In the slides, each result is converted to **±1** and then averaged:
- Same bits ($00$ or $11$) → $+1$  
- Different bits ($01$ or $10$) → $-1$

So the correlation is just the **average of the product**:
$$
E(a,b)=\langle A\cdot B\rangle=\frac{1}{N}\sum_{k=1}^{N} A_k B_k
$$

The counts formula above is **exactly the same thing**, just written using totals of each outcome.

---

In [ ]:
def calculate_correlations(measurements):
    """
    Calculate E(a,b) for each basis pair.
    
    Args:
        measurements: Dict from organize_measurements_by_basis()
    
    Returns:
        Dict mapping (alice_basis, bob_basis) to correlation value
    """
    correlations = {}
    
    for basis_pair, results in measurements.items():
        total = sum(results.values()) # Total measurements: results['00'] + results['01'] + results['10'] + results['11']
        
        if total > 0: # to avoid division by zero :p
              
            # Formula: (Same - Different) / Total
            E = 0 # replace with formula ...
            correlations[basis_pair] = E
        else:
            correlations[basis_pair] = 0
    
    # Print for debugging
    print("Correlations:")
    for basis, corr in correlations.items():
        print(f"  E{basis} = {corr:.4f}")
    
    return correlations


---
## 7) Calculate CHSH Value

CHSH combines the four correlations in a **specific pattern** that exposes quantum behavior.

**Formula:** $S = E(a_1,b_1) - E(a_1,b_2) + E(a_2,b_1) + E(a_2,b_2)$

With our bases:
- $a_1 = 0^\circ$, $a_2 = 90^\circ$ (Alice)
- $b_1 = 45^\circ$, $b_2 = 135^\circ$ (Bob)

Exercise: Implement `calculate_chsh_value()`

In [3]:
def calculate_chsh_value(correlations, alice_bases=ALICE_BELL_BASES, bob_bases=BOB_BELL_BASES):
    """
    Calculate the CHSH Bell parameter S.

    Args:
        correlations: Dictionary mapping (alice_basis, bob_basis) to correlation value E(a, b).
        example:
        correlations[(a1, b1)] = E(a1, b1)
        alice_bases: List of Alice's two measurement bases (default: ['0', '90']).
        bob_bases: List of Bob's two measurement bases (default: ['45', '135']).

    
    Returns:
        float: |S| value
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'

    # Note: correlations[(a1, b1)] gives E(a1, b1)
    
    # TODO: Calculate S using the CHSH formula
    # S = E(a1,b1) - E(a1,b2) + E(a2,b1) + E(a2,b2)
    S = 0  # Replace with actual calculation
    
    return abs(S)


def check_bell_inequality(chsh_value):
    """Returns True if Bell inequality is violated (quantum entanglement detected)."""
    return chsh_value > BELL_INEQUALITY_THRESHOLD

## Visualization helper (provided)
It is not part of the protocol logic. Use it to see correlations and the CHSH value clearly.

In [ ]:
def visualize_bell_test_results(correlations, chsh_value, title="Bell Test Results"):
    fig = plt.figure(figsize=(12, 5))
    
    # Correlation heatmap
    ax1 = fig.add_subplot(121)
    alice_bases = sorted(set(b[0] for b in correlations.keys()))
    bob_bases = sorted(set(b[1] for b in correlations.keys()))
    corr_matrix = np.zeros((len(alice_bases), len(bob_bases)))
    
    for i, a_base in enumerate(alice_bases):
        for j, b_base in enumerate(bob_bases):
            if (a_base, b_base) in correlations:
                corr_matrix[i, j] = correlations[(a_base, b_base)]
    
    colors = [(0.8, 0.2, 0.2), (1, 1, 1), (0.2, 0.2, 0.8)]
    cmap = LinearSegmentedColormap.from_list('rwb', colors, N=100)
    
    im = ax1.imshow(corr_matrix, cmap=cmap, vmin=-1, vmax=1)
    ax1.set_title('Correlation Values E(a,b)')
    ax1.set_xticks(np.arange(len(bob_bases)))
    ax1.set_yticks(np.arange(len(alice_bases)))
    ax1.set_xticklabels([f"{b}°" for b in bob_bases])
    ax1.set_yticklabels([f"{a}°" for a in alice_bases])
    ax1.set_xlabel("Bob's Angle")
    ax1.set_ylabel("Alice's Angle")
    
    for i in range(len(alice_bases)):
        for j in range(len(bob_bases)):
            ax1.text(j, i, f"{corr_matrix[i, j]:.2f}", ha="center", va="center",
                    color="black" if abs(corr_matrix[i, j]) < 0.5 else "white")
    
    fig.colorbar(im, ax=ax1)
    
    # CHSH bar chart
    ax2 = fig.add_subplot(122)
    ax2.bar([0], [chsh_value], width=0.4, color='purple', alpha=0.7)
    ax2.axhline(y=2.0, color='r', linestyle='-', label='Classical Limit (2.0)')
    ax2.axhline(y=2*np.sqrt(2), color='b', linestyle='--', label='Quantum Limit (2√2)')
    ax2.set_ylim(0, 3.0)
    ax2.set_xticks([0])
    ax2.set_xticklabels(['CHSH Value'])
    ax2.text(0, chsh_value + 0.1, f"{chsh_value:.3f}", ha='center')
    ax2.legend(loc='upper left')
    ax2.set_title('CHSH Value vs Limits')
    
    verdict = " Entanglement!" if chsh_value > 2 else " No quantum"
    ax2.text(0, 1.0, verdict, ha='center', fontsize=14, fontweight='bold',
             color='green' if chsh_value > 2 else 'red')
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---
## Run the Complete Bell Test!

Let's put it all together and verify quantum entanglement.

In [ ]:
# RUN THE COMPLETE BELL TEST

# Create 1000 Bell pairs (Alice and Bob share each pair)
NUM_PAIRS = 400
print(f"\nCreating {NUM_PAIRS} Bell pairs...")
bell_pairs = [create_bell_pair_singlet_state() for _ in range(NUM_PAIRS)]

# Run measurements on all pairs
print("Running measurements...")
results, alice_bases_used, bob_bases_used = run_bell_test_measurements(bell_pairs)

# Organize by basis pairs for correlation calculation
print("\nOrganizing results...")
organized = organize_measurements_by_basis(results, alice_bases_used, bob_bases_used)

# Calculate correlations
print("\nCalculating correlations...")
correlations = calculate_correlations(organized)

# Calculate CHSH
chsh = calculate_chsh_value(correlations)
print(f"\n" + "="*60)
print(f"CHSH VALUE: {chsh:.4f}")
print("="*60)

if check_bell_inequality(chsh):
    print("\n QUANTUM ENTANGLEMENT DETECTED!")
    print("   The Bell inequality is VIOLATED.")
    print("   This cannot be explained by classical physics!")
else:
    print("\n No quantum correlations detected.")
    print("   Check your implementation.")

In [ ]:
# Visualize
visualize_bell_test_results(correlations, chsh, "Bell Test: Singlet State |Ψ-⟩")

---
## Challenge: Detect Eavesdropping!

When Eve intercepts and measures the qubits, she destroys the entanglement.
So she must **send a replacement qubit** to Bob (otherwise he would notice missing photons).
In this simplified model, she **measures**, then **recreates a new product state** that matches her measurement result.

Let's see how this affects the CHSH value.


In [ ]:
def create_eavesdropped_state(bell_qc):
    """
    Simulate Eve measuring the Bell pair (destroys entanglement).
    """
    # Eve measures in Z basis
    eve_qc = bell_qc.copy()
    eve_qc.measure_all()
    eve_result = run_circuit(eve_qc, shots=1)
    eve_bits = list(eve_result.keys())[0]
    
    # Eve recreates a product state based on her measurement
    qc = QuantumCircuit(2)
    if eve_bits[0] == '1':  # qubit 1
        qc.x(1)
    if eve_bits[1] == '1':  # qubit 0
        qc.x(0)
    
    return qc

## Test with Eve intercepting 70% of pairs

In [ ]:
# Test with Eve intercepting 70% of pairs
EVE_PERCENTAGE = 0.7

print(f"\n EAVESDROPPING SIMULATION")
print(f"Eve intercepts {EVE_PERCENTAGE*100}% of Bell pairs\n")

# Create Bell pairs
print(f"Create Bell {NUM_PAIRS} pairs ...")
bell_pairs = [create_bell_pair_singlet_state() for _ in range(NUM_PAIRS)]

# Eve compromises some pairs
compromised = int(NUM_PAIRS * EVE_PERCENTAGE)
print(f"Number of compromised {compromised} / {NUM_PAIRS} ...")
eve_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]

# Run Bell test on compromised pairs
print("Run circuit and get measurements ...")
results, alice_bases_used, bob_bases_used = run_bell_test_measurements(eve_pairs)

# Organize, calculate correlations and CHSH
organized = organize_measurements_by_basis(results, alice_bases_used, bob_bases_used)
correlations = calculate_correlations(organized)
chsh = calculate_chsh_value(correlations)

print(f"\nCHSH VALUE with Eve: {chsh:.4f}")

if check_bell_inequality(chsh):
    print("\n QUANTUM CORRELATIONS DETECTED!")
    print("   The system is entangled. The channel appears secure.")
    if EVE_PERCENTAGE > 0:
        print(f"   (Note: Eve was present at {EVE_PERCENTAGE*100}%, but didn't intercept enough to break the inequality.)")
else:
    print("\n EAVESDROPPING DETECTED!")
    print("   The Bell inequality is satisfied (S ≤ 2.0).")
    print("   Alice and Bob should abort: the connection is NOT secure.")


 EAVESDROPPING SIMULATION
Eve intercepts 70.0% of Bell pairs



NameError: name 'NUM_PAIRS' is not defined

In [ ]:

visualize_bell_test_results(correlations, chsh, f"Bell Test: {EVE_PERCENTAGE*100}% Eavesdropped")

---
## Summary

**What we learned:**
1. Created entangled Bell states
2. Applied measurement basis transformations
3. Calculated correlations E(a,b)
4. Computed the CHSH value S
5. Verified |S| > 2 proves quantum entanglement
6. Eavesdropping destroys entanglement → detectable!

---

**Next:** Use your functions for the **E91 quantum key distribution protocol**!

==> Open `02_E91_Protocol.ipynb` to continue!

*(Your functions will be imported directly from this notebook)*